In [2]:
# Importando bibliotecas necessárias

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import os

In [3]:
# Funções gerais do projeto

def extract_bans(bans):
    res_dic = {
        "Bans": [],
        "Picks": [],
    }
    
    teams_ban = bans.split(";")
    for ban in teams_ban:
        content = ban.strip().split(" ")
        if content[0] == "C9":
            if content[1] == "ban":
                res_dic["Bans"].append(content[2])
            else:
                res_dic["Picks"].append(content[2])
    
    return res_dic

In [4]:
# Começando análises da Cloud9

c9url = "https://www.vlr.gg/team/matches/188/cloud9"

res = requests.get(c9url)

soup = bs(res.text, 'html.parser')

In [5]:
# Escolhendo jogos apenas do VCT Americas (mais recente)

matches = soup.findAll('a', attrs = {'class': 'wf-card fc-flex m-item'})

match_links = []

for match in matches:
    camp = match.find('div', attrs = {'class': 'text-of'}).text.strip().split(":")[0]
    if camp == "VCT 2023":
    	match_links.append(c9url + match.get('href'))

In [25]:
# Entrando em cada jogo e salvando as informações que desejamos em um arquivo json em uma pasta do time

if not os.path.exists("cloud9"):
    os.mkdir("cloud9")

for link in match_links:
    opponent = link.find('div', attrs = {'class': 'wf-title-med mod-single'}).text.strip()
    print(f"Raspando informações do jogo: Cloud9 vs {opponent}")
    
    r = requests.get(link)
    soup = bs(r.text, 'html.parser')
    
    bans = soup.find('div', attrs = {'class': 'match-header-note'})
    c9_bans = extract_bans(bans)
    
    c9_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
    opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
    
    games = int(c9_score) + int(opponent_score)
    
    first_map_id = int(soup.find('div', {'class': 'vm-stats-gamesnav-item js-map-switch'}).get('data-game-id'))
    
    maps = {}
    for gid in range(first_map_id, first_map_id+games):
        print(f"Raspando informações do mapa de id: {gid}")
        
        map_url = f"{link}/?game={gid}&tab=overview"
        r = requests.get(map_url)
        soup = bs(r.text, 'html.parser')
        
        map_name = soup.find('div', attrs = {'class': 'map'}).find_all('span')[0].text.strip()
        
        c9_rounds = soup.find('div', attrs = {'class': 'score mod-win'}).text.strip()
        opponent_rounds = soup.find('div', attrs = {'class': 'score '}).text.strip()
        
        team = soup.find('div', {'class': 'vm-stats-game mod-active'}).find_all('table')[0].find('tbody').find_all('tr')
        